In [29]:
import transformers
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer
# from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
# import evaluate
import torch
import numpy as np

from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [31]:
# load dataset
dataset = load_dataset("csv", data_files="data/atcs-final-test.csv")

Generating train split: 2 examples [00:00, 156.50 examples/s]


In [32]:
#Set up HuggingFace Pipeline with Llama-2-7b-chat-hf model
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.float16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
)

#LLM intialized in HuggingFace Pipeline wrapper
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]


In [34]:
# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    llm.resize_token_embeddings(len(tokenizer))

In [35]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [36]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/2 [00:00<?, ? examples/s]


KeyError: 'text'